In [50]:
import gurobipy as gp
import numpy
import plotly.graph_objects as go
from gurobipy import GRB

In [51]:
COLUMNAS = ["a", "b", "c", "d", "e", "f", "g", "h"]
FILAS = [1, 2, 3, 4, 5, 6, 7, 8]
CASILLAS = [f"{columna}{fila}" for columna in COLUMNAS for fila in FILAS]
PIEZAS = ["K", "R", "Q", "N", "B"]

In [52]:
def ataque_torre(casilla: str) -> list[str]:
    columna_torre = COLUMNAS.index(casilla[0]) + 1
    fila_torre = int(casilla[1])
    casillas_atacadas = []
    for fila in range(1, 9):
        if fila != fila_torre:
            casillas_atacadas.append(f"{COLUMNAS[columna_torre - 1]}{fila}")
    for i, columna in enumerate(COLUMNAS):
        if i + 1 != columna_torre:
            casillas_atacadas.append(f"{columna}{fila_torre}")
    return casillas_atacadas


def ataque_alfil(casilla: str) -> list[str]:
    columna_alfil = COLUMNAS.index(casilla[0])
    fila_alfil = int(casilla[1])
    casillas_atacadas = []
    direcciones = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
    for dx, dy in direcciones:
        x, y = columna_alfil, fila_alfil
        while 0 <= x + dx < 8 and 0 <= y + dy <= 8:
            x += dx
            y += dy
            if 0 <= x < 8 and 1 <= y <= 8:
                casillas_atacadas.append(f"{COLUMNAS[x]}{y}")
    return casillas_atacadas


def ataque_dama(casilla: str) -> list[str]:
    columna_dama = COLUMNAS.index(casilla[0])
    fila_dama = int(casilla[1])
    casillas_atacadas = []
    for i in range(8):
        if i != columna_dama:
            casillas_atacadas.append(f"{COLUMNAS[i]}{fila_dama}")
        if i + 1 != fila_dama:
            casillas_atacadas.append(f"{COLUMNAS[columna_dama]}{i + 1}")

    direcciones = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
    for dx, dy in direcciones:
        x, y = columna_dama, fila_dama
        while 0 <= x + dx < 8 and 0 <= y + dy <= 8:
            x += dx
            y += dy
            if 0 <= x < 8 and 1 <= y <= 8:
                casillas_atacadas.append(f"{COLUMNAS[x]}{y}")
    return list(set(casillas_atacadas))


def ataque_rey(casilla: str) -> list[str]:
    columna_rey = COLUMNAS.index(casilla[0])
    fila_rey = int(casilla[1])
    movimientos = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
    casillas_atacadas = []
    for dx, dy in movimientos:
        nueva_columna = columna_rey + dx
        nueva_fila = fila_rey + dy
        if 0 <= nueva_columna < 8 and 1 <= nueva_fila <= 8:
            casilla_atacada = f"{COLUMNAS[nueva_columna]}{nueva_fila}"
            casillas_atacadas.append(casilla_atacada)
    return casillas_atacadas


def ataque_caballo(casilla: str) -> list[str]:
    columna_caballo = COLUMNAS.index(casilla[0])
    fila_caballo = int(casilla[1])
    movimientos = [(-2, -1), (-2, 1), (-1, -2), (1, -2), (-1, 2), (1, 2), (2, -1), (2, 1)]
    casillas_atacadas = []
    for dx, dy in movimientos:
        nueva_columna = columna_caballo + dx
        nueva_fila = fila_caballo + dy
        if 0 <= nueva_columna < 8 and 1 <= nueva_fila <= 8:
            casilla_atacada = f"{COLUMNAS[nueva_columna]}{nueva_fila}"
            casillas_atacadas.append(casilla_atacada)
    return casillas_atacadas

In [53]:
ATAQUE = {"K": ataque_rey, "R": ataque_torre, "Q": ataque_dama, "N": ataque_caballo, "B": ataque_alfil}
GANANCIA_PIEZA = {"K": 14, "R": 28, "Q": 28, "N": 7, "B": 16}

In [54]:
model = gp.Model("max_chess")

In [55]:
X = model.addVars(PIEZAS, CASILLAS, name="X", vtype=GRB.BINARY)

In [56]:
model.setObjective(gp.quicksum(GANANCIA_PIEZA[pieza] * X[pieza, casilla] for pieza in PIEZAS for casilla in CASILLAS), GRB.MAXIMIZE)

In [57]:
for pieza in PIEZAS:
    model.addConstr(gp.quicksum(X[pieza, casilla] for casilla in CASILLAS) >= 1, f"Place_{pieza}")

for casilla in CASILLAS:
    model.addConstr(gp.quicksum(X[pieza, casilla] for pieza in PIEZAS) <= 1, f"Spot_{casilla}")

for pieza in PIEZAS:
    for casilla in CASILLAS:
        CASILLAS_ATACADAS_PIEZAS = ATAQUE[pieza](casilla)
        for casilla_atacada in CASILLAS_ATACADAS_PIEZAS:
            for pieza_ in PIEZAS:
                model.addConstr(X[pieza_, casilla_atacada] <= 1 - X[pieza, casilla], f"Attack_{pieza}_{casilla}_{pieza_}_{casilla_atacada}")

In [58]:
model.setParam("TimeLimit", 60)

Set parameter TimeLimit to value 60


In [59]:
model.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 18409 rows, 320 columns and 37320 nonzeros
Model fingerprint: 0xea2976f1
Variable types: 0 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 226.0000000
Presolve removed 16999 rows and 0 columns
Presolve time: 0.26s
Presolved: 1410 rows, 320 columns, 17519 nonzeros
Variable types: 0 continuous, 320 integer (320 binary)
Found heuristic solution: objective 239.0000000

Root relaxation: objective 4.090369e+02, 1550 iterations, 0.23 seconds (0.25 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl | 

In [60]:
TABLERO = {casilla: pieza for pieza in PIEZAS for casilla in CASILLAS if X[pieza, casilla].X > 0.9}

In [61]:
PIEZAS_ICONOS = {"K": "♚", "R": "♜", "Q": "♛", "N": "♞", "B": "♝"}

In [62]:
model.objVal

280.0

In [63]:
# Crear el tablero de ajedrez
tablero = numpy.zeros((8, 8), dtype=int)
tablero[1::2, ::2] = 1
tablero[::2, 1::2] = 1


# Crear la figura del tablero usando Heatmap
fig = go.Figure()

fig.add_trace(
    go.Heatmap(
        z=tablero,
        colorscale=["#eeeed2", "#769656"],
        showscale=False,
        x=COLUMNAS,
        y=list(map(str, FILAS)),
        hoverinfo="none",
    )
)

for casilla, pieza in TABLERO.items():
    x = COLUMNAS.index(casilla[0])
    y = 7 - FILAS.index(int(casilla[1]))
    fig.add_annotation(x=x, y=y, text=PIEZAS_ICONOS[pieza], showarrow=False, font_size=50, font_color="black")


fig.update_layout(coloraxis_showscale=False)
fig.update_layout(height=800, width=800, title_text=f"Utilidad: {model.objVal}", template="ggplot2")
fig.show()